<a href="https://colab.research.google.com/github/Felikin/Proyecto-Regresion-Lineal/blob/main/Proyecto_1_Machine_Learning_Felipe_Gonz%C3%A1lez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto 1 Regresión Lineas
### Felipe González Urrego


In [39]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import r2_score


## Punto 1


### Carga de datos

In [40]:
from google.colab import files
uploaded = files.upload()

Saving COLcovidData05062021.csv to COLcovidData05062021 (1).csv


In [41]:
datos = pd.read_csv("COLcovidData05062021.csv")
datos.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,0.026,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


Escojemos 3 paises, Noruega, Canadá y Palestina

### Selección de Países

En este bloque seleccionamos 3 países: Noruega, Canadá y Palestina. Para esto se crea una función donde se envía la base de datos y el código del país como parámetro y esta devuelve el dataframe con el país seleccionado filtrado.

En este bloque solo utilizo la funcion `escoger_pais` para mostrar que filtra correctamente los datos del país seleccionado.

In [42]:
def escoger_pais(codigo_pais, data):
  pais = data[data["iso_code"] == codigo_pais]
  return pais

nor = escoger_pais("NOR", datos)  
can = escoger_pais("CAN", datos)  
pse = escoger_pais("PSE", datos)  

print(nor.head())
print(pse.head())
print(can.head())


      iso_code continent  ... life_expectancy human_development_index
62829      NOR    Europe  ...            82.4                   0.957
62830      NOR    Europe  ...            82.4                   0.957
62831      NOR    Europe  ...            82.4                   0.957
62832      NOR    Europe  ...            82.4                   0.957
62833      NOR    Europe  ...            82.4                   0.957

[5 rows x 59 columns]
      iso_code continent  ... life_expectancy human_development_index
64725      PSE      Asia  ...           74.05                   0.708
64726      PSE      Asia  ...           74.05                   0.708
64727      PSE      Asia  ...           74.05                   0.708
64728      PSE      Asia  ...           74.05                   0.708
64729      PSE      Asia  ...           74.05                   0.708

[5 rows x 59 columns]
      iso_code      continent  ... life_expectancy human_development_index
15120      CAN  North America  ...     

### Funciones polinómicas, Gaussianas y sigmoides.

In [43]:
def polyFBase(obs,M):
  idx = np.linspace(0,M-1,M)
  phi_x = np.power(obs,idx)
  return phi_x

def expFBase(obs,M):
  mu_s = np.linspace(0,500.,M)
  s2 = 0.05
  phi_x = np.zeros_like(mu_s)
  for m in range(0,len(mu_s)):
    phi_x[m] = np.exp(-((obs-mu_s[m])**2)/(2*s2))
  phi_x[0] = 1.
  return phi_x

def sigFBase(obs, M):
  mu_s = np.linspace(0,500.,M)
  s = 0.05
  phi_x = np.zeros_like(mu_s)
  for m in range(0,len(mu_s)):
    phi_x[m] = 1/(1+(np.exp(-(obs-mu_s[m])/(s))))
  phi_x[0] = 1.
  return phi_x

### Creación de la Matriz


In [44]:
def matrixPHIDesign(x_vect,M,fbase):
  N = len(x_vect)
  mPHI = np.zeros((N,M))
  for n in range(0,N):
    x_n = x_vect[n]
    if fbase == 'poly':
      vectPhi_xn = polyFBase(x_n,M)
    if fbase == 'exp':
      vectPhi_xn = expFBase(x_n,M)
    if fbase == "sig":
      vectPhi_xn = sigFBase(x_n,M)
    mPHI[n,:] = vectPhi_xn
  return mPHI

In [45]:
from ipywidgets import interact
%matplotlib inline

### Regresión Lineal Máxima Verosimilitud.

En este punto decidí crear una manera interactiva al igual que lo visto en clase. Donde es posible elegir: clase funciones base a usar, número de funciones base, etiquetas o el tipo de caso que se quiere analizar, si desea usar regularización, valor del factor de regularización y el país que se quiere analizar dentro de los 3 escogidos. Los detalles serán comentados en el código.

In [46]:
@interact(TipoCaso=["new_cases","new_deaths","new_deaths_per_million","new_cases_per_million"],numFBase = (3,2000,2),TipoFbase = ["exp","poly", "sig"],flagRegularize = [True,False], regParam =(0,0.1,0.0001), pais = ["PSE", "NOR", "CAN"])
def get_DataCOVID(TipoCaso,numFBase,TipoFbase,flagRegularize,regParam, pais):
  # Capturamos date, total_cases, new_cases, total_deaths, new_deaths, total_cases_per_million, total_deaths_per_million
  
  #Llamamos la función creada bloques atrás para seleccionar los datos del pais correspondiente.
  covid_pais = escoger_pais(pais,datos)
  #Filtramos los NA, con un nuevo Dataframe.
  df = covid_pais[covid_pais[TipoCaso].notna()]
  #Es necesario guardar la fecha con el nuevo df porque al remover NA's las dimensiones cambian del original cambian.
  fecha = df["date"]
  x = np.linspace(0,len(fecha),len(fecha))
  if TipoCaso == 'new_cases':
    t = df['new_cases']
  elif TipoCaso == 'new_deaths':
    t = df['new_deaths']
  elif TipoCaso == 'new_deaths_per_million':
    t = df['new_deaths_per_million']
  elif TipoCaso == 'new_cases_per_million':
    t = df['new_cases_per_million']
  
  plt.plot(x,t)
  plt.title(TipoCaso)
  plt.show()

  # Construirmos el modelo de regresion para el COVID
  x = np.squeeze(x)
  t = np.squeeze(t)
  N = len(x)
  tObs = t
  # La matriz de diseño PHI será
  matPHI = matrixPHIDesign(x ,M = numFBase,fbase=TipoFbase)
  matPHIT = matPHI.T

  # Calculamos el w de máxima verosimilitud
  # Parmámetro de regularización (previene el sobre entrenamiento)
  if flagRegularize == True:
    print(regParam)
    w_ml = np.linalg.inv(matPHIT.dot(matPHI)+regParam*np.eye(numFBase))@(matPHIT@tObs)
  else:
    w_ml = np.linalg.inv(matPHIT.dot(matPHI))@(matPHIT@tObs)

  # Realizamos la estimación de las etiquetas
  yEst = matPHI@w_ml

  plt.figure()
  plt.plot(x,tObs,'-b')
  plt.plot(x,yEst,'--r')
  plt.xlabel('x (obs)')
  plt.ylabel('t (etiquetas)')
  plt.legend(('t obs','t estimado'))
  plt.show()
  # Calculamos el error MSE de predicción
  aux = (tObs-matPHI@w_ml)
  error = 0.5*(aux.T)@aux
  print("Error de predicción es: ",error)
  print('R2 de la función: ',r2_score(t,yEst))

  # Ahora hagamos una predicción un paso adelante para el siguiente mes
  x_obs_new = np.linspace(456,500,50)
  phi_vec_new = matrixPHIDesign(x_obs_new,numFBase,TipoFbase)
  y_new_obs = phi_vec_new@w_ml

  plt.figure()
  plt.plot(x,tObs,'-b')
  plt.plot(x,yEst,'--r')
  #Graficamos el nuevo punto
  plt.plot(x_obs_new,y_new_obs,'dy')
  plt.xlabel('x (obs)')
  plt.ylabel('t (etiquetas)')
  plt.legend(('t obs','t estimado'))


interactive(children=(Dropdown(description='TipoCaso', options=('new_cases', 'new_deaths', 'new_deaths_per_mil…

####Discusiones

Los resultados de las funciones sigmoide y gaussianas se ajustan muy bien a los valores observados; cuando el número de funciones base es alto. Mientras que las polinomiales se deben revisar con pocas funciones base ya que al ponerle un número mayor de 10 sufre un sobre-entrenamiento. El error de predicción disminuye a medida que incrementa el número de funciones base y es aun menor si se emplean funciones sigmoidales, llegando a ser en algunos casos de 1 orden de magnitud. Los modelos con funciones gaussianas se ajustan bien para cualquier caso estudiado dentro de la base de datos, al igual que para cualquier país en los 3 seleccionados. El coeficiente de correlación $r^2$ aumenta a medida que se incrementan el número de funciones base, su valor es de $0.66$ para pocas funciones. Esto indica un buen ajuste a pesar de ser pocas funciones base. Nos entrega un valor muy cercano a $1$ con un alto número de funciones base, lo que indica un buen ajuste del modelo.

### Regresión Lineal Bayesiana

In [49]:
@interact(TipoCaso=["new_cases","new_deaths","new_deaths_per_million","new_cases_per_million"],numFBase = (3,2000,2),TipoFbase = ["exp","poly", "sig"], pais = ["PSE", "NOR", "CAN"], alpha = (1,10,1), lambda_v =(0,0.1,0.0001))
def get_DataCOVID(TipoCaso,numFBase,TipoFbase, pais, alpha,lambda_v):
  # Capturamos date, total_cases, new_cases, total_deaths, new_deaths, total_cases_per_million, total_deaths_per_million
  bayesian = BayesianRidge(tol=1e-6, fit_intercept=False, compute_score=True)
  bayesian.set_params(alpha_init=alpha, lambda_init=lambda_v)
  lw=2
  #Llamamos la función creada bloques atrás para seleccionar los datos del pais correspondiente.
  covid_pais = escoger_pais(pais,datos)
  #Filtramos los NA, con un nuevo Dataframe.
  df = covid_pais[covid_pais[TipoCaso].notna()]
  #Es necesario guardar la fecha con el nuevo df porque al remover NA's las dimensiones cambian del original cambian.
  fecha = df["date"]
  x = np.linspace(0,len(fecha),len(fecha))
  if TipoCaso == 'new_cases':
    t = df['new_cases']
  elif TipoCaso == 'new_deaths':
    t = df['new_deaths']
  elif TipoCaso == 'new_deaths_per_million':
    t = df['new_deaths_per_million']
  elif TipoCaso == 'new_cases_per_million':
    t = df['new_cases_per_million'] 

  plt.plot(x,t)
  plt.title(TipoCaso)
  plt.show()

  if TipoFbase == "exp":
    phi_x = matrixPHIDesign(x,numFBase,TipoFbase)
  elif TipoFbase == "poly":
    phi_x = matrixPHIDesign(x,numFBase,TipoFbase)
  elif TipoFbase == "sig":
    phi_x = matrixPHIDesign(x,numFBase,TipoFbase)
  
  print()
  bayesian.fit(phi_x,t) 
  yest = bayesian.predict(phi_x)
  
  aux = (t-yest)
  error = 0.5*(aux.T)@aux
  print("Error de predicción es: ",error)
  print('R2 de la función: ',r2_score(t,yest))
  
  plt.plot(x, yest, "--r",
             label="t est")
  plt.plot(x, t, "-b",
         label="t obs")
  plt.ylabel("t etiquetas")
  plt.xlabel("Feature X")
  plt.legend(loc="upper right")
  plt.show()


interactive(children=(Dropdown(description='TipoCaso', options=('new_cases', 'new_deaths', 'new_deaths_per_mil…

#### Discusiones

Al igual que en el punto anterior las sigmoide y gaussianas se ajustan muy bien al modelo. Así mismo tenemos la misma condición con las funciones polinomiales. Al igual que en el caso anterior el $r^2$ da cercano a 1 para un alto número de funciones base. Sin embargo el error de predicción es de 1 orden de magnitud mayor que en el caso anterior con igual número de funciones base. Esto probablemente se pueda mejorar modificando los parámetros del framework. Aún así por el costo computacional y desempeño se recomienda la regresión anterior para este caso particular.

## Punto 2

Usamos los datos `Medical Cost Personal Dataset`

In [ ]:
uploaded = files.upload()

Saving insuranceDB.csv to insuranceDB (1).csv


In [ ]:
costos = pd.read_csv("insuranceDB.csv")
costos.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### Observaciones y etiquetas

Como observaciones tomaremos `age`,`sex`,`bmi`,`children`,`smoker` y `region`. Las etiquetas serán `charges`.

In [ ]:
from sklearn.metrics import r2_score

def poly(X,M):
  N,D = X.shape
  PHI_X = np.zeros((N,D*M))
  Xtrain = X.to_numpy()
  for n in range(0,N):
    x_obs = Xtrain[n,:]
    phi_x = []
    for x_obs_d in x_obs:
      idx = np.linspace(0,M,M)
      phi_x.append(np.power(x_obs_d,idx))
    phi_x = np.squeeze(phi_x).reshape(M*D)
    PHI_X[n,:] = phi_x
  return PHI_X


def exp(X,M):
  N,D = X.shape
  s2=0.05
  PHI_X = np.zeros((N,D*M))
  Xtrain = X.to_numpy()
  # Calculamos el Rango de cada característica sobre el dataset
  minsDB = X.min(axis=0) 
  maxsDB = X.max(axis=0)
  for n in range(0,N):
    x_obs = Xtrain[n,:]
    phi_x = []
    count = 0
    for x_obs_d in x_obs:
      mus = np.linspace(minsDB[count],maxsDB[count],M)
      aux = []
      for m in range(0,M):
        aux.append(np.exp(-((x_obs_d - mus[m])**2)/(2*s2))) # Completar
      phi_x.append(aux)
    phi_x = np.squeeze(phi_x).reshape(M*D)
    PHI_X[n,:] = phi_x
  return PHI_X

def sig(X,M):
  N,D = X.shape
  s=0.05
  PHI_X = np.zeros((N,D*M))
  Xtrain = X.to_numpy()
  # Calculamos el Rango de cada característica sobre el dataset
  minsDB = X.min(axis=0) 
  maxsDB = X.max(axis=0)
  for n in range(0,N):
    x_obs = Xtrain[n,:]
    phi_x = []
    count = 0
    for x_obs_d in x_obs:
      mus = np.linspace(minsDB[count],maxsDB[count],M)
      aux = []
      for m in range(0,M):
        aux.append(1/(1 + np.exp(-(x_obs_d - mus[m])/s))) # Completar
      phi_x.append(aux)
    phi_x = np.squeeze(phi_x).reshape(M*D)
    PHI_X[n,:] = phi_x
  return PHI_X

In [ ]:
costos["sex"]=costos["sex"].astype("category")
costos["sex_cat"]= costos["sex"].cat.codes
costos["smoker"]=costos["smoker"].astype("category")
costos["smoker_cat"]= costos["smoker"].cat.codes
costos["region"]=costos["region"].astype("category")
costos["region_cat"]= costos["region"].cat.codes

###Regresion Bayesiana

In [ ]:
@interact(numFBase = (1,300,1),tipoFbase = ["exp","poly", "sig"], alpha = (1,10,1), lambda_v =(0,0.1,0.0001))
def get_charges(numFBase,tipoFbase, alpha,lambda_v):
  x = costos.loc[:,["age","bmi","children","smoker_cat","region_cat","sex_cat"]]
  reg = BayesianRidge(tol=1e-6,fit_intercept=False, compute_score=True)
  if tipoFbase == "poly":
    obs = poly(x,numFBase) 
  elif tipoFbase == "exp":
    obs = exp(x,numFBase)
  elif tipoFbase == "sig":
    obs = sig(x,numFBase)
  reg.set_params(alpha_init = alpha, lambda_init = lambda_v)
  reg.fit(obs, costos["charges"])
  yest=reg.predict(obs)
  x["charges"]= costos["charges"]
  x["charges_pred"]=yest
  aux = (costos["charges"]-yest)
  error = 0.5*(aux.T)@aux

  print('R2 de la función: ',r2_score(costos["charges"],yest))
  print("Error de predicción: ",error )
  print(x)

interactive(children=(IntSlider(value=150, description='numFBase', max=300, min=1), Dropdown(description='tipo…

#### Discusiones.

Para este caso el modelo requiere de muchas funciones base para ajustarse correctamente, las cuales no pueden ser implementadas por costos computacionales. Vemos que la que mejor desempeño tiene es la gaussiana, llegando a un $r^2$ de $0.20$. Las que peor desempeño presentan son las funciones polinomiales. El error de predicción es grande, y no cambia significativamente con el número de funciones bases, ni con el tipo de funciones usadas.